## Formatera data från SCAAR så output kan laddas till DosReg

Gå till https://www.ucr.uu.se/swedeheart/ och logga in med Siths-kort\
Gå till "Rapporter"\
Välj "Export till Excel Angio-PCI"\
Rapportdatum: T.ex 2024-01-01 - 2025-01-01\
Angio/PCI: Enbart angio\
Procedur/Segment: Procedur\
Klicka på "Beställ" och ladda ner resulterande excel-fil.\
Öppna excel-fil, radera första 3 kolumnerna "Personnr eller motsv", "Typ av personnummer" och "Födelsedatum" och spara till input_data.


In [ ]:
import pandas as pd

# Läs in data från SCAAR

SCAAR_data_path = "C:/Projekt/GIT/rvbrtg/Data/input_data/Angio_2024.xlsx"
data = pd.read_excel(SCAAR_data_path)

#data.head()

In [3]:
#Minska ner tabellen och döp om kolumner

data_subset = data[["Kön", "Ålder vid procedur", "Längd (cm)", "Vikt (kg)", "Angiograför", "Punktionställe", "Labnamn", "Stråldos (µGym2)", "Genomlysningstid (h:mm:ss)", ]].copy()

data_subset.columns = ["Sex", "Age", "Length_cm", "Weight_kg", "Operator", "Accesspoint", "Lab", "KAP_uGym2", "Fluorotime_h_mm_ss"]

data_subset.head()

,Sex,Age,Length_cm,Weight_kg,Operator,Accesspoint,Lab,KAP_uGym2,Fluorotime_h_mm_ss
0,Kvinna,87,160.0,67.0,"Hasslow, Jacob",A radialis höger,Lab 1,"422,00",00:05:05
1,Man,86,172.0,90.0,"Pettersson, Björn",A radialis vänster,Lab 1,"1490,00",00:05:37
2,Kvinna,86,NaN,NaN,"Andersson, Jonas",A femoralis konverterad från radialis,Lab 1,"152,00",00:03:05
3,Man,86,171.0,75.0,"Hagström, Henrik",A radialis höger,Lab 1,"749,00",00:04:03
4,Man,85,NaN,NaN,"Pettersson, Björn",A radialis höger,Lab 1,"1040,00",00:06:55


In [4]:
#Kolla datatyper. Är KAP numeriskt eller en sträng

data_subset.dtypes

Sex                    object
Age                     int64
Length_cm             float64
Weight_kg             float64
Operator               object
Accesspoint            object
Lab                    object
KAP_uGym2              object
Fluorotime_h_mm_ss     object
dtype: object

In [5]:
#Ersätt , med . samt byt från sträng till float och konvertera från uGym2 till Gycm2
data_subset["KAP_uGym2"] = data_subset["KAP_uGym2"].replace(',','.',regex=True).astype(float)
data_subset["KAP_uGym2"] = data_subset["KAP_uGym2"] * 0.01

data_subset.rename(columns = {"KAP_uGym2":"KAP_Gycm2"}, inplace = True)

#data_subset.dtypes
data_subset.head()

,Sex,Age,Length_cm,Weight_kg,Operator,Accesspoint,Lab,KAP_Gycm2,Fluorotime_h_mm_ss
0,Kvinna,87,160.0,67.0,"Hasslow, Jacob",A radialis höger,Lab 1,4.22,00:05:05
1,Man,86,172.0,90.0,"Pettersson, Björn",A radialis vänster,Lab 1,14.90,00:05:37
2,Kvinna,86,NaN,NaN,"Andersson, Jonas",A femoralis konverterad från radialis,Lab 1,1.52,00:03:05
3,Man,86,171.0,75.0,"Hagström, Henrik",A radialis höger,Lab 1,7.49,00:04:03
4,Man,85,NaN,NaN,"Pettersson, Björn",A radialis höger,Lab 1,10.40,00:06:55


In [6]:
#Printa ut antal per kön och medelvärde för KAP för att skriva in i DosReg-mall

print(data_subset.groupby("Sex").size())

print(data_subset.groupby("Sex").mean(numeric_only = True))

Sex
Kvinna    202
Man       385
dtype: int64
              Age   Length_cm  Weight_kg  KAP_Gycm2
Sex                                                
Kvinna  69.747525  162.784173  71.707792   4.308196
Man     67.659740  177.532609  88.934932   7.607411
